# Detailed Exploration of Universal Gate Set for Intermediate Zeta

## Header

In [1]:
import numpy as np
import QDYN
from QDYN.shutil import mkdir
from QDYN.pulse import Pulse, pulse_tgrid
from QDYN.gate2q import Gate2Q
from QDYN.linalg import norm, vectorize
from os.path import join
from notebook_utils import (
    get_zeta_table, plot_zeta_data, filter_table, get_logical_energies_table,
    get_stage4_table, bare_decomposition, get_transition_table, J_target)
from mgplottools.io import writetotxt
from select_for_stage4 import GATE_RE, GATE_IM, GATE
from generate_zeta_universal import write_config
from analytical_pulses import AnalyticalPulse
from run_oct import propagate
from collections import OrderedDict
from zeta_systematic_variation import systematic_variation
import pandas as pd

In [2]:
H_left = GATE['H_left']
def write_target_gate_dat(rf, gate):
    writetotxt(join(rf, 'target_gate.dat'), GATE_RE[gate], GATE_IM[gate])

In [3]:
def J_PE(U):
    if U is None:
        return None
    else:
        C = U.closest_unitary().concurrence()
        max_loss = np.max(1.0 - U.logical_pops())
        return J_target('PE', C, max_loss)

def F_avg_H_left(U):
    if U is None:
        return None
    else:
        return U.F_avg(H_left)

In [4]:
from multiprocessing import Process
import os
def system(cmd):
    p = Process(target=os.system, args=(cmd,))
    p.start()
    return p

In [5]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [6]:
get_zeta_table = QDYN.memoize.memoize(get_zeta_table)
get_zeta_table.load('zeta_table.cache')

In [7]:
zeta_table = get_zeta_table('./runs_050_RWA', T=50)

In [8]:
w_max = 1.0 # minimum frequency in the rotating frame to be resolved. (for calculation of nt)

In [9]:
n_qubit = 5; n_cavity = 6

## Analysis

In [14]:
w1 = 6.00 # GHz
w2 = 6.32 # GHz
wc = 5.75 # GHZ
root = './runs_zeta_detailed/w2_%dMHz_wc_%dMHz' % (w2*1000, wc*1000)
mkdir(root)

* what is the the entangling energy?

In [16]:
zeta = float(filter_table(zeta_table, [(w2, wc),])['zeta [MHz]']); zeta

-16.716000000000008

In [17]:
T_entangling = 500.0 / abs(zeta); T_entangling # ns

29.911462072266076

* Verify the PE. What do the logical eigenstates look like? What are the dressed qubit levels? What required spectral resolution do we anticipate?

In [18]:
T = T_entangling
rf = join(root, 'verify_PE'); mkdir(rf)
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
pulse.write(join(rf, 'pulse.guess'))
U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True)
print "concurrence: %.5f" % U.concurrence()

concurrence: 1.00000


In [19]:
psi = {}; psi['00'], psi['01'], psi['10'], psi['11'] \
= np.genfromtxt(join(rf, 'logical_states.dat'), unpack=True, usecols=range(4))
for label in ['00', '01', '10', '11']:
    print("%s = %s" % (label, bare_decomposition(psi[label], n_qubit, n_cavity)))

00 = 100.0% {000}
01 = 1.5% {001} + 98.4% {010}
10 = 5.7% {001} + 0.3% {010} + 94.0% {100}
11 = 33.1% {020} + 1.1% {101} + 65.8% {110}


In [20]:
transition_table = get_transition_table(filter_table(zeta_table, [(w2, wc),]))
transition_table.reset_index(drop=True)

,E01 - E00 [MHz],E10 - E00 [MHz],E11 - E10 [MHz],E11 - E01 [MHz],bare w2 [MHz],bare w1 [MHz]
0,6328.69,6017.286,6311.974,6000.57,6320,6000


In [21]:
w1_dressed0 = transition_table['E10 - E00 [MHz]'][0] / 1000.0 # GHz
w1_dressed1 = transition_table['E11 - E01 [MHz]'][0] / 1000.0 # GHz
w2_dressed0 = transition_table['E01 - E00 [MHz]'][0] / 1000.0 # GHz
w2_dressed1 = transition_table['E11 - E10 [MHz]'][0] / 1000.0 # GHz

In [22]:
print("Separation of dressed frequencies, left qubit: %.3f MHz"
      % abs(float(transition_table['E11 - E01 [MHz]']) - float(transition_table['E10 - E00 [MHz]'])))
print("Separation of dressed frequencies, right qubit: %.3f MHz"
      % abs(float(transition_table['E11 - E10 [MHz]']) - float(transition_table['E01 - E00 [MHz]'])))

Separation of dressed frequencies, left qubit: 16.716 MHz
Separation of dressed frequencies, right qubit: 16.716 MHz


In [23]:
print "Spectral resolution of pulse: %.1f MHz" % pulse.dw(freq_unit='MHz')

Spectral resolution of pulse: 33.4 MHz


* Verify the local gate

In [24]:
T = 2*T_entangling
rf = join(root, 'verify_SQ'); mkdir(rf)
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
pulse.write(join(rf, 'pulse.guess'))
print "Spectral resolution of pulse: %.1f MHz" % pulse.dw(freq_unit='MHz')
U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True)
print "concurrence: %.5f" % U.concurrence()

Spectral resolution of pulse: 16.7 MHz
concurrence: 0.00007


In [25]:
T = 4*T_entangling
rf = join(root, 'verify_SQ_2cycles'); mkdir(rf)
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=6.0, gate="CPHASE", J_T='SM', prop_guess='T')
pulse = Pulse(tgrid=pulse_tgrid(T, nt), time_unit='ns',ampl_unit='MHz')  # zero amplitude
pulse.write(join(rf, 'pulse.guess'))
print "Spectral resolution of pulse: %.1f MHz" % pulse.dw(freq_unit='MHz')
U = propagate(rf, 'pulse.guess', rwa=True, keep=True, force=True)
print "concurrence: %.5f" % U.concurrence()

Spectral resolution of pulse: 8.4 MHz
concurrence: 0.00014


* Can we compress the PE by adding a pulse and OCT?

In [ ]:
compression = 0.9
wd = 0.5*(w1+w2)
rf = join(root, 'compress_PE_%.1f' % compression); mkdir(rf)

In [ ]:
rf

In [ ]:
# write runfolder
T = compression*T_entangling
nt = int(max(2000, 100 * w_max * T))
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="CPHASE", J_T='PE', prop_guess='F')

In [ ]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 200, 500]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=J_PE, n_procs=40)

In [ ]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [ ]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True, keep=None)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

In [ ]:
proc = system("./run_oct.py --target=PE --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [ ]:
proc.join(); U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

In [ ]:
U.concurrence() - U_guess.concurrence()

Note: 0.7 and 1.2 also don't work

* Can we implement $H_L$ at $T = 2 T_\pi$ starting from field-free pulse?

In [26]:
rf = join(root, '1cycle_H_left'); mkdir(rf)
wd = 0.5*(w1+w2)

In [27]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/1cycle_H_left'

In [28]:
T = 2*T_entangling
nt = int(max(2000, 100 * w_max * T))
write_target_gate_dat(rf, 'H_left')
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="target_gate.dat", J_T='SM', prop_guess='F')

In [32]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 100, 150]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=F_avg_H_left, n_procs=40)

,E0,w_L,fig_of_merit
27,150,6.000000,0.193467
31,150,6.000570,0.195021
15,50,6.328690,0.199310
0,0,6.000000,0.199350
1,0,6.320000,0.199350
2,0,6.160000,0.199350
3,0,6.017286,0.199350
4,0,6.000570,0.199350
5,0,6.008928,0.199350
6,0,6.328690,0.199350


In [33]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 150.0, 'T': T, 'w_L': 6.000000, 'w_d': 6.000000},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [34]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

Concurrence: 0.350875
Loss: 0.053989


In [46]:
proc = system("./run_oct.py --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [47]:
proc.join(); U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

 9356  2.208135483561E-01  1.002229345243E-07  0.000000000000E+00  2.20813648579
 9357  2.208133505027E-01  1.001959723938E-07  0.000000000000E+00  2.20813450698
 9358  2.208131527031E-01  1.001690247013E-07  0.000000000000E+00  2.20813252872
 9359  2.208129549572E-01  1.001420914360E-07  0.000000000000E+00  2.20813055099
 9360  2.208127572650E-01  1.001151725875E-07  0.000000000000E+00  2.20812857380
 9361  2.208125596265E-01  1.000882681448E-07  0.000000000000E+00  2.20812659714
 9362  2.208123620416E-01  1.000613780976E-07  0.000000000000E+00  2.20812462103
 9363  2.208121645104E-01  1.000345024349E-07  0.000000000000E+00  2.20812264544
 9364  2.208119670327E-01  1.000076411462E-07  0.000000000000E+00  2.20812067040
 9365  2.208117696086E-01  9.998079422073E-08  0.000000000000E+00  2.20811869589


In [48]:
U.concurrence() - U_guess.concurrence()

-0.51605610399905855

* Can we implement $H_L$ at $T = T_\pi$ starting from field-free pulse?

In [52]:
rf = join(root, 'half_cycle_H_left'); mkdir(rf)
wd = 0.5*(w1+w2)

In [53]:
rf

'./runs_zeta_detailed/w2_6320MHz_wc_5750MHz/half_cycle_H_left'

In [38]:
T = T_entangling
nt = int(max(2000, 100 * w_max * T))
write_target_gate_dat(rf, 'H_left')
write_config(join(rf, 'config'), T, nt, wc, w2, wd=wd, gate="target_gate.dat", J_T='SM', prop_guess='F')

In [42]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 0.0, 'T': T, 'w_L': wd, 'w_d': wd},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
systematic_variation(rf, pulse, vary=OrderedDict([
            ('E0', [0.0, 50, 150, 200]), 
            ('w_L', [w1, w2, 0.5*(w1+w2),  
                     w1_dressed0, w1_dressed1, 0.5*(w1_dressed0+w1_dressed1),
                     w2_dressed0, w2_dressed1, 0.5*(w2_dressed0+w2_dressed1)])]),
            fig_of_merit=F_avg_H_left, n_procs=40)

,E0,w_L,fig_of_merit
18,150,6.000000,0.188577
22,150,6.000570,0.189322
31,200,6.000570,0.197611
27,200,6.000000,0.201381
32,200,6.008928,0.206737
35,200,6.320332,0.236371
28,200,6.320000,0.236377
34,200,6.311974,0.239808
33,200,6.328690,0.240044
19,150,6.320000,0.261705


In [43]:
pulse = AnalyticalPulse('1freq_rwa', T, nt,
            parameters={'E0': 150.0, 'T': T, 'w_L': 6.000000, 'w_d': 6.000000},
            t0=0.0, time_unit='ns', ampl_unit='MHz', freq_unit='MHz', mode="complex")
pulse.write(join(rf, 'pulse_guess.json'))

In [44]:
U_guess = propagate(rf, 'pulse_guess.json', rwa=True, force=True)
print("Concurrence: %f" % U_guess.closest_unitary().concurrence())
print("Loss: %f" % U_guess.pop_loss())

Concurrence: 0.999956
Loss: 0.064700


In [54]:
proc = system("./run_oct.py --rwa --pre-simplex='pulse_guess.json' {rf}".format(rf=rf))

In [55]:
proc.join(); U = propagate(rf, 'pulse.dat', rwa=True, force=True);
! tail {rf}/oct_iters.dat | cut -b 1-80

 9356  2.208135483561E-01  1.002229345243E-07  0.000000000000E+00  2.20813648579
 9357  2.208133505027E-01  1.001959723938E-07  0.000000000000E+00  2.20813450698
 9358  2.208131527031E-01  1.001690247013E-07  0.000000000000E+00  2.20813252872
 9359  2.208129549572E-01  1.001420914360E-07  0.000000000000E+00  2.20813055099
 9360  2.208127572650E-01  1.001151725875E-07  0.000000000000E+00  2.20812857380
 9361  2.208125596265E-01  1.000882681448E-07  0.000000000000E+00  2.20812659714
 9362  2.208123620416E-01  1.000613780976E-07  0.000000000000E+00  2.20812462103
 9363  2.208121645104E-01  1.000345024349E-07  0.000000000000E+00  2.20812264544
 9364  2.208119670327E-01  1.000076411462E-07  0.000000000000E+00  2.20812067040
 9365  2.208117696086E-01  9.998079422073E-08  0.000000000000E+00  2.20811869589


In [56]:
U.concurrence() - U_guess.concurrence()

-0.51605610399905855